# TRAINING: DeBERTa, early_stopping_patience = 4

In [1]:
import numpy as np
import pandas as pd
import time

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

import datasets
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import EarlyStoppingCallback, IntervalStrategy

2024-04-04 14:03:06.269680: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-04 14:03:07.275285: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-04-04 14:03:07.275367: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-04-04 14:03:07.275374: W tensorfl

In [2]:
model_name = "microsoft/deberta-v3-large"

tokenizer = AutoTokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/milos.kosprdic.ivi/.local/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:455: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
with open('./SciFact_train.csv', 'rb') as fh:
    df_train = pd.read_csv(fh)
df_train.drop('Unnamed: 0', axis=1, inplace=True)
df_train

,claim,abstract,label
0,Angiotensin converting enzyme inhibitors are a...,Renal considerations in angiotensin converting...,1
1,Reducing H3k4me3 methylation induces mouse epi...,MLL1 Inhibition Reprograms Epiblast Stem Cells...,1
2,Expression of oncolytic virus antigens as pept...,Detecting and targeting tumor relapse by its r...,0
3,Varenicline monotherapy is more effective afte...,Combination varenicline and bupropion SR for t...,2
4,Acute ablation of Snail in the embryonic corte...,Control of Apoptosis by Asymmetric Cell Divisi...,0
...,...,...,...
966,"Higher plasma levels of I-carnitine, when asso...",Intestinal microbiota metabolism of L-carnitin...,1
967,Women with a higher birth weight are more like...,Birth Size and Breast Cancer Risk: Re-analysis...,1
968,In domesticated populations of Saccharomyces c...,Domestication and Divergence of Saccharomyces ...,0
969,Active Ly49Q induces neutrophil polarization.,The Ly49Q receptor plays a crucial role in neu...,1


In [4]:
with open('./SciFact_valid.csv', 'rb') as fh:
    df_valid = pd.read_csv(fh)
df_valid.drop('Unnamed: 0', axis=1, inplace=True)
df_valid

,claim,abstract,label
0,MICAL redox enzymes regulate actin dynamics in...,Drosophila MICAL regulates myofilament organiz...,1
1,The cytokine receptor which regulates chemotax...,Myeloperoxidase acts as a profibrotic mediator...,0
2,RUNX1 is downregulated or mutated in TLX1 T-ALL.,Reverse engineering of TLX oncogenic transcrip...,1
3,There is no relation between Erythromycin use ...,Use of macrolides in mother and child and risk...,2
4,Risk of cardiovascular events can be cut by a ...,Cardiovascular protection with antihypertensiv...,1
...,...,...,...
115,Genes involved in pre-mRNA splicing have a sig...,A genome-wide siRNA screen reveals diverse cel...,1
116,Metastatic colorectal cancer treated with a si...,Chemotherapy options in elderly and frail pati...,2
117,Lack of clpC does not affect sporulation effic...,High-Throughput Genetic Screens Identify a Lar...,0
118,GATA3 regulates cell cycle progression in bone...,GATA-3 regulates hematopoietic stem cell maint...,1


In [7]:
with open('./SciFact_test.csv', 'rb') as fh:
    df_test = pd.read_csv(fh)
df_test.drop('Unnamed: 0', axis=1, inplace=True)
df_test

,claim,abstract,label
0,Women are more susceptible to death due to pne...,Gender-dependent differences in outcome after ...,1
1,Cellular clocks are not predictive of mitosis ...,Robust synchronization of coupled circadian an...,2
2,Cells undergoing methionine restriction may ac...,microRNAs: A Safeguard against Turmoil? Emergi...,0
3,Replacement of OCT4 and SOX2 genes with GATA3 ...,Reprogramming of human fibroblasts to pluripot...,0
4,IgA plasma cells that are specific for transgl...,High abundance of plasma cells secreting trans...,2
...,...,...,...
117,Charcoal shows no benefit for acute paraquat p...,Effect of activated charcoal hemoperfusion on ...,1
118,The risk of breast cancer among parous women d...,Pregnancy characteristics and maternal risk of...,2
119,Inositol lipid 3-phosphatase PTEN converts Ptd...,"PTEN Regulates PI(3,4)P2 Signaling Downstream ...",1
120,Combination nicotine replacement therapies wit...,Combination varenicline and bupropion SR for t...,1


In [7]:
df_train['text'] = tokenizer.cls_token + df_train['claim'] + tokenizer.sep_token + df_train['abstract'] + tokenizer.sep_token
df_train.drop(columns=['claim', 'abstract'], inplace=True)
df_train_robertaL

,label,text
0,1,[CLS]Angiotensin converting enzyme inhibitors ...
1,1,[CLS]Reducing H3k4me3 methylation induces mous...
2,0,[CLS]Expression of oncolytic virus antigens as...
3,2,[CLS]Varenicline monotherapy is more effective...
4,0,[CLS]Acute ablation of Snail in the embryonic ...
...,...,...
966,1,"[CLS]Higher plasma levels of I-carnitine, when..."
967,1,[CLS]Women with a higher birth weight are more...
968,0,[CLS]In domesticated populations of Saccharomy...
969,1,[CLS]Active Ly49Q induces neutrophil polarizat...


In [10]:
df_valid['text'] = tokenizer.cls_token + df_valid['claim'] + tokenizer.sep_token + df_valid['abstract'] + tokenizer.sep_token
df_valid.drop(columns=['claim', 'abstract'], inplace=True)

df_test['text'] = tokenizer.cls_token + df_test['claim'] + tokenizer.sep_token + df_test['abstract'] + tokenizer.sep_token
df_test.drop(columns=['claim', 'abstract'], inplace=True)

In [12]:
konacno_train = Dataset.from_pandas(df_train)
konacno_valid = Dataset.from_pandas(df_valid)
konacno_test = Dataset.from_pandas(df_test)

In [12]:
my_dataset_dict = datasets.DatasetDict({
    'train': konacno_train,
    'valid': konacno_valid,
    'test': konacno_test
})

In [3]:
def preprocess_function(examples):
    max_input_length = 512
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=max_input_length)

In [16]:
tokenized = my_dataset_dict.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [4]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [5]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [6]:
def compute_metrics(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='weighted')
    precision = precision_score(y_true=labels, y_pred=pred, average='weighted')
    f1 = f1_score(y_true=labels, y_pred=pred, average='weighted')    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [21]:
training_args = TrainingArguments(
    output_dir="./SCIFACT_deberta_early4",
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=15,
    save_total_limit = 5,
    weight_decay=0.01,
    evaluation_strategy=IntervalStrategy.STEPS,
    eval_steps = 50,
    #save_strategy="epoch",
    metric_for_best_model = 'f1',
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['valid'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
)

In [22]:
start_time = time.time()
trainer.train()
total_time = time.time()-start_time

/home/milos.kosprdic.ivi/.local/lib/python3.7/site-packages/transformers/optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,No log,1.064412,0.500000,0.394088,0.500000,0.439178
100,No log,0.978910,0.533333,0.468738,0.533333,0.462983
150,No log,0.808694,0.700000,0.561105,0.700000,0.619164
200,No log,0.823899,0.708333,0.553501,0.708333,0.621076
250,No log,0.811878,0.691667,0.541890,0.691667,0.607684
300,No log,0.843940,0.683333,0.567405,0.683333,0.604465
350,No log,1.189721,0.583333,0.464774,0.583333,0.499988
400,No log,0.610991,0.725000,0.579398,0.725000,0.641019
450,No log,0.634677,0.800000,0.828800,0.800000,0.793913
500,0.840100,0.507479,0.850000,0.848339,0.850000,0.848261


/home/milos.kosprdic.ivi/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/milos.kosprdic.ivi/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/milos.kosprdic.ivi/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/milos.kosprdic.ivi/.local/lib/python3

In [23]:
predictions = trainer.predict(tokenized['test'])

In [24]:
classifications = []
for pred in predictions[0]:
    classifications.append(np.argmax(pred))

In [25]:
target_names = ['NO_EVIDENCE', 'SUPPORT', 'CONTRADICT']
print(classification_report(tokenized["test"]['label'], classifications, target_names=target_names))

              precision    recall  f1-score   support

 NO_EVIDENCE       0.88      0.95      0.91        44
     SUPPORT       0.90      0.88      0.89        51
  CONTRADICT       0.88      0.78      0.82        27

    accuracy                           0.89       122
   macro avg       0.88      0.87      0.88       122
weighted avg       0.89      0.89      0.88       122



In [26]:
confusion_matrix(tokenized["test"]['label'], classifications)

array([[42,  1,  1],
       [ 4, 45,  2],
       [ 2,  4, 21]])

In [27]:
trainer.save_model("DeBERTa_early4")